# TwitterTrends-1-TrendsToFile

## Importing libraries

In [1]:
import $ivy.`org.twitter4j:twitter4j-core:4.0.6`

import $ivy.`com.typesafe.play::play-json:2.6.2`

import $ivy.`org.apache.commons:commons-io:1.3.2`

import $ivy.$                                   


import $ivy.$                                   


import $ivy.$                                    

In [2]:
import twitter4j.conf.ConfigurationBuilder
import twitter4j.Twitter
import twitter4j.TwitterFactory
import twitter4j.Location

import scala.collection.JavaConverters._
import scala.util.control.Breaks._
import scala.collection.mutable.ListBuffer

import java.io._
import java.util.Calendar
import java.net.URLEncoder
import java.text.SimpleDateFormat

import play.api.libs.json._

import org.apache.commons.io.FileUtils

import twitter4j.conf.ConfigurationBuilder

import twitter4j.Twitter

import twitter4j.TwitterFactory

import twitter4j.Location


import scala.collection.JavaConverters._

import scala.util.control.Breaks._

import scala.collection.mutable.ListBuffer


import java.io._

import java.util.Calendar

import java.net.URLEncoder

import java.text.SimpleDateFormat


import play.api.libs.json._


import org.apache.commons.io.FileUtils

## Setting Twitter API Keys

In [ ]:
val consumerKey = "YOU_CONSUMER_KEY"
val consumerSecret = "YOU_CONSUMER_SECRET"
val accessToken = "YOU_ACCESS_TOKEN"
val accessTokenSecret = "YOU_TOKEN_SECRET"

*Note: If you are not using Jupyter better load this tokens in a [more proper way](https://danielasfregola.com/2015/06/01/loading-configurations-in-scala/) and use a code like this in order to load the properties:*

```Scala
def loadProperties() {
  val consumerKey = ConfigFactory.load().getString("twitter.consumer.key")
  val consumerSecret = ConfigFactory.load().getString("twitter.consumer.secret")
  val accessToken = ConfigFactory.load().getString("twitter.access.key")
  val accessTokenSecret = ConfigFactory.load().getString("twitter.access.secret")
}

loadProperties()
```

## Instantiate objects and variables

*getTwitterInstance* will load the Twitter API Keys we just have set and will create the [TwitterFactory](http://twitter4j.org/javadoc/twitter4j/TwitterFactory.html), which will allow us to comunicate with the [Twitter API](https://dev.twitter.com/overview/api).

In [ ]:
def getTwitterInstance: Twitter = {
  val cb = new ConfigurationBuilder()
  cb.setDebugEnabled(true)
    .setOAuthConsumerKey(consumerKey)
    .setOAuthConsumerSecret(consumerSecret)
    .setOAuthAccessToken(accessToken)
    .setOAuthAccessTokenSecret(accessTokenSecret)
  return new TwitterFactory(cb.build()).getInstance
}

Set all variables needed in the process:

In [5]:
val twitter = getTwitterInstance
var availableTrends : twitter4j.ResponseList[twitter4j.Location] = null
var placeTrends : twitter4j.Trends = null
val SEP = "|"
var key = ""
var numTowns = 0
val MAX_NUM_TOWNS = 5
var placeName = ""
var woeid = ""
var countryName = ""
var countryNameURL = ""
var cityName = ""
var cityNameURL = ""
var response = ""
var responseJson : JsValue = null
var lon = ""
var lat = ""
var trendsDate = ""
val trendsDateFormat = new SimpleDateFormat("HH:mm:ss|dd/MM/yyyy") // date format for the 'date' field included in textfile
val filenameDateFormat = new SimpleDateFormat("_YYYY-MM-dd-HHmmss") // date format for the filename
val keyDateFormat = new SimpleDateFormat("YYYYMMddHHmmss") // date format for the 'key' field included in textfile
var trendsList = new ListBuffer[String]()
var trends = ""
var trendLine = ""
var fDateString = filenameDateFormat.format(Calendar.getInstance().getTime())
var keyDateString = keyDateFormat.format(Calendar.getInstance().getTime())
val nL = sys.props("line.separator")

twitter: Twitter = TwitterImpl{INCLUDE_MY_RETWEET=PostParameter{name='include_my_retweet', value='true', file=null, fileBody=null}}
availableTrends: twitter4j.ResponseList[Location] = null
placeTrends: twitter4j.Trends = null
SEP: String = "|"
key: String = ""
numTowns: Int = 0
MAX_NUM_TOWNS: Int = 5
placeName: String = ""
woeid: String = ""
countryName: String = ""
countryNameURL: String = ""
cityName: String = ""
cityNameURL: String = ""
response: String = ""
responseJson: JsValue = null
lon: String = ""
lat: String = ""
trendsDate: String = ""
trendsDateFormat: SimpleDateFormat = java.text.SimpleDateFormat@72dd7cbc
filenameDateFormat: SimpleDateFormat = java.text.SimpleDateFormat@6da3d6ee
keyDateFormat: SimpleDateFormat = java.text.SimpleDateFormat@b184d160
trendsList: ListBuffer[String] = ListBuffer()
trends: String = ""
trendLine: String = ""
fDateString: String = "_2017-08-10-234037"
keyDateString: String = "20170810234037"
nL: String = """


"""

## Define functions

In [6]:
// Returns the top 10 trending topics for a specific WOEID
def myGetPlaceTrends(woeid: String) : String = {
    placeTrends = twitter.trends.getPlaceTrends(woeid.toInt)
    trendsDate = trendsDateFormat.format(placeTrends.getTrendAt())
    trendsList = new ListBuffer[String]()

    for (trend <- placeTrends.getTrends()) {
        trendsList += trend.getName()
    }

    trends = trendsDate + SEP + (trendsList.mkString(","))
    return trends
}

// Returns the response from the called webservice
@throws(classOf[java.io.IOException])
def get(url: String) : String = scala.io.Source.fromURL(url, "UTF-8").mkString


// Returns specific location's trendic topic info: country, city, woeid, longitude and latitude 
def getLocationData(location: Location) : String = {
    woeid = location.getWoeid.toString
    countryName = location.getCountryName
    countryNameURL = URLEncoder.encode(countryName, "UTF-8")
    cityName = location.getName
    cityNameURL = URLEncoder.encode(cityName, "UTF-8")
    lon = ""
    lat = ""

    try {
        response = get("http://nominatim.openstreetmap.org/search?country=" + countryNameURL + "&city=" + cityNameURL + "&limit=1&format=json")
        responseJson = Json.parse(response.drop(1).dropRight(1)) // delete the first ([) and last char (] returned by Twitter)
        lon = responseJson("lon").toString.replace("\"", "") // delete " chars returned by Twitter
        lat = responseJson("lat").toString.replace("\"", "") // delete " chars returned by Twitter
    } catch {
        case ioe: java.io.IOException => return("ERROR: Got java.io.IOException: " + ioe) // TODO: Exception handling
        case ste: java.net.SocketTimeoutException => return("ERROR: Got java.net.SocketTimeoutException: " + ste) // TODO: Exception handling
    }

    return countryName + SEP + cityName + SEP + woeid + SEP + lon + SEP + lat
}


// Twitter's response has 
def isValidTown(location: Location) : Boolean = {
    placeName = location.getPlaceName
    if (placeName.equals("Town")) return true
    else return false
}

defined function myGetPlaceTrends
defined function get
defined function getLocationData
defined function isValidTown

## Main program

Now we will do a loop in which, every 10 minutes, we obtain the top trending topic from 5 random cities. 

The 2 main methods are:
- [getAvailableTrends()](http://twitter4j.org/javadoc/twitter4j/api/TrendsResources.html#getAvailableTrends--), [available](https://dev.twitter.com/rest/reference/get/trends/available) in Twitter API: Returns the locations that Twitter has trending topic information for.
- [getPlaceTrends(int woeid)](http://twitter4j.org/javadoc/twitter4j/api/TrendsResources.html#getPlaceTrends-int-), [place](https://dev.twitter.com/rest/reference/get/trends/place) in Twitter API: Returns the top 10 trending topics for a specific WOEID, if trending information is available for it.

The rest of the code is basically some data quality and formatting strings.

*Note: You cat set the loop time as you want, but take care about limit usage. Both methods are **rate limited** as stated in "Resource Information" section from Twitter documentation*.

In [ ]:
while (true){
    println("Start batch")
    
    fDateString = filenameDateFormat.format(Calendar.getInstance().getTime()) // get current date with desired format
    var writer = new PrintWriter(new File("../datasets/trendFiles/tmp/trends" + fDateString + ".csv")) // open new file for writing

    numTowns = 0

    // Returns the locations that Twitter has trending topic information for
    availableTrends = twitter.trends.getAvailableTrends()

    // The next location loop must be breakable (we must break once we find 5 cities)
    breakable {
        for (location <- scala.util.Random.shuffle(availableTrends.asScala)) {
            trendLine = ""

            // We will only get 5 towns
            if (numTowns < MAX_NUM_TOWNS) {
               if (isValidTown(location) == true) {
                    keyDateString = keyDateFormat.format(Calendar.getInstance().getTime())
                   // create a unique key based on date and some other info
                    key = (numTowns + 1).toString + "-" + keyDateString + "-" + java.util.UUID.randomUUID.toString
                    trendLine += key + ";" + getLocationData(location)
                    // Write only valid info with no error
                    if (!trendLine.contains("ERROR:")){
                        trendLine += SEP + myGetPlaceTrends(woeid)
                        writer.write(trendLine + nL)
                        numTowns += 1
                    }
                }
            } else {
                break
            }
        }
    }
    
    writer.flush
    writer.close

    FileUtils.copyFileToDirectory(new File("../datasets/trendFiles/tmp/trends" + fDateString + ".csv"), new File("../datasets/trendFiles/"))
    FileUtils.forceDelete(new File("../datasets/trendFiles/tmp/trends" + fDateString + ".csv"))

    println("Finish batch, waiting 10 min till next one...")
    Thread.sleep(600000) //wait 10 minutes
}

Start batch
Finish batch, waiting 10 min till next one...
Start batch
Finish batch, waiting 10 min till next one...
Start batch
Finish batch, waiting 10 min till next one...
